In [272]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageEnhance
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import cv2

'2.3.0-tf'

In [203]:
ROOT_PATH = "/Users/huylv/Desktop/Master CS Messina/Year 2/ATDA/Steel Defect Detection/data/Sample/NEU-DET"
IMAGES_PATH = "/Users/huylv/Desktop/Master CS Messina/Year 2/ATDA/Steel Defect Detection/data/Sample/NEU-DET/IMAGES"
ANNOTATIONS_PATH = "/Users/huylv/Desktop/Master CS Messina/Year 2/ATDA/Steel Defect Detection/data/Sample/NEU-DET/ANNOTATIONS"
IMAGES_DIR = os.listdir(IMAGES_PATH)
ANNOTATIONS_DIR = os.listdir(ANNOTATIONS_PATH)
images_arr = []
annotations_arr = []
for image in IMAGES_DIR:
    images_arr.append(IMAGES_PATH + "/" + image)
for annotation in ANNOTATIONS_DIR:
    annotations_arr.append(ANNOTATIONS_PATH + "/" + annotation)

images_arr.sort()
annotations_arr.sort()

In [204]:
class BoundingBox:
    def __init__(self, _xmin, _ymin, _xmax, _ymax):
        self.xmin = _xmin
        self.ymin = _ymin
        self.xmax = _xmax
        self.ymax = _ymax

In [205]:
class Image:
    def __init__(self, _name, _boundingBox, _height = 200, _weight = 200):
        self.name = _name
        self.boundingBox = _boundingBox
        self.height = _height
        self.weight = _weight
        
    def toString(self):
        return "name: {}\nbox: {}".format(self.name, self.boundingBox)

In [206]:
def loadingAnnotationImage(annotation):
    xmlFile = open(annotation)
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    filename = root.find('filename').text
#     for obj in root.iter('object'):
    boundBoxes = []
    for boundingObject in root.findall('object'):
        xmin = boundingObject.find('bndbox').find('xmin').text
        ymin = boundingObject.find('bndbox').find('ymin').text
        xmax = boundingObject.find('bndbox').find('xmax').text
        ymax = boundingObject.find('bndbox').find('ymax').text
        box = BoundingBox(xmin, ymin, xmax, ymax)
        boundBoxes.append(box)
    image = Image(filename, boundBoxes)
    return image

In [207]:
image_bounded_arr = []
for annotation in annotations_arr:
    image_bounded_arr.append(loadingAnnotationImage(annotation))

In [237]:
data_image = []
targets_image = []
filenames_image = []
for i in range(len(image_bounded_arr)):
    name = image_bounded_arr[i].name
    for box in image.boundingBox:
        xmin = float(box.xmin) / image.weight
        ymin = float(box.ymin) / image.height
        xmax = float(box.xmax) / image.weight
        ymax = float(box.ymax) / image.height
        loading_image = load_img(images_arr[i], target_size = (224, 224))
        loading_image = img_to_array(loading_image)
        data_image.append(loading_image)
        targets_image.append((xmin, ymin, xmax, ymax))
        filenames_image.append(name)
        
data_image = np.array(data_image, dtype="float32") / 255.0
targets_image = np.array(targets_image, dtype="float32")
# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
split = train_test_split(data_image, targets_image, filenames_image, test_size=0.2, random_state = 42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[:2]
(trainFilenames, testFilenames) = split[:2]
print("trainImages.size: {}\ttestImages.size:{}".format(len(trainImages), len(testImages)))
print("trainTargets.size: {}\ttestTargets.size:{}".format(len(trainTargets), len(testTargets)))
print("trainFilenames.size: {}\ttestFilenames.size:{}".format(len(trainFilenames), len(testFilenames)))
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")
TEST_FILE_PATH = ROOT_PATH + "/Test.txt"
f = open(TEST_FILE_PATH, "w")
f.write("{}".format(testFilenames))
f.close()

trainImages.size: 1440	testImages.size:360
trainTargets.size: 1440	testTargets.size:360
trainFilenames.size: 1440	testFilenames.size:360
[INFO] saving testing filenames...


In [233]:
print(trainImages.shape)
print(testImages.shape)
print(trainTargets.shape)
print(testTargets.shape)
print(trainFilenames.shape)
print(testFilenames.shape)

(1440, 224, 224, 3)
(360, 224, 224, 3)
(1440, 224, 224, 3)
(360, 224, 224, 3)
(1440, 224, 224, 3)
(360, 224, 224, 3)


In [264]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 32

# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights = "imagenet", include_top = False,
            input_tensor = Input(shape = (224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(6, activation="softmax")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs = vgg.input, outputs = bboxHead)

# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(lr = LEARNING_RATE)
# model.compile(loss = "mse", optimizer = opt)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = opt, metrics = ['accuracy'])
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
history = model.fit(trainImages, trainTargets, validation_data = (testImages, testTargets), 
              batch_size = BATCH_SIZE, epochs = NUM_EPOCHS, verbose = 1)

# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(ROOT_PATH + "/model.h5", save_format = "h5")
# plot the model training history
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

ValueError: in user code:

    /Users/huylv/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/huylv/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/huylv/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/huylv/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/huylv/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /Users/huylv/anaconda3/lib/python3.7/site-packages/tensorflow/py

In [276]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 32
opt = Adam(lr = LEARNING_RATE, decay = LEARNING_RATE / NUM_EPOCHS)

baseModel = tf.keras.applications.ResNet152V2(weights='imagenet')
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (7, 7))(headModel)
headModel = Flatten(name = "flatten")(headModel)
headModel = Dense(256, activation = "relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(6, activation = "softmax")(headModel)
model = Model(inputs = baseModel.input, outputs = headModel)

for layer in baseModel.layers:
    layer.trainable = False
    
model.compile(loss = "sparse_categorical_crossentropy", optimizer = opt, metrics = ['accuracy'])

history = model.fit(trainImages, trainTargets, validation_data = (testImages, testTargets), epochs = NUM_EPOCHS)
# H = model.fit_generator(
# 	trainGen,
# 	steps_per_epoch=totalTrain // config.BS,
# 	validation_data=valGen,
# 	validation_steps=totalVal // config.BS,
# 	epochs=config.NUM_EPOCHS)
# fit(
#     x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None,
#     validation_split=0.0, validation_data=None, shuffle=True, class_weight=None,
#     sample_weight=None, initial_epoch=0, steps_per_epoch=None,
#     validation_steps=None, validation_batch_size=None, validation_freq=1,
#     max_queue_size=10, workers=1, use_multiprocessing=False
# )

ValueError: Input 0 of layer average_pooling2d_1 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 1000]

In [268]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised = True, with_info = True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

(train_set, valid_set, test_set), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:70%]', 'train[:15%]', 'train[:10%]'],
    with_info = True,
    as_supervised=True,
)
print(dataset_size)
print(class_names)
print(n_classes)



Dataset tf_flowers downloaded and prepared to /Users/huylv/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
3670
['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
5
